<a href="https://colab.research.google.com/github/M4r1nh0/classifica-o-binaria-com-pytorch-com-deep-learning---breast-cancer--02-/blob/master/Projeto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install 'skorch==0.7.0'
!pip install 'torch==1.4.0'

     |████████████████████████████████| 112kB 3.0MB/s 
     |████████████████████████████████| 753.4MB 19kB/s 
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101


In [ ]:
import pandas as pd #Panda pro dataset
import numpy as np #numpy pros vetores
import torch.nn as nn #torch por neural network
from skorch import NeuralNetBinaryClassifier 
import torch
import skorch
import torch.nn.functional as F #trabalhamos com função de ativação
from sklearn.model_selection import GridSearchCV #cross validation


In [ ]:
torch.__version__
skorch.__version__

'0.7.0'

In [ ]:
np.random.seed(123)
torch.manual_seed(123)

In [ ]:
previsores = pd.read_csv('entradas_breast.csv')
classes = pd.read_csv('saidas_breast.csv')

In [ ]:
previsores = np.array(previsores, dtype= 'float32')
classes = np.array(classes, dtype= 'float32').squeeze(1)

In [ ]:
previsores.shape

(569, 30)

In [ ]:
classes.shape

(569,)

In [ ]:
class classificador_torch(nn.Module):
  def __init__(self, activation, neurons, initializer):
    super().__init__()
    self.dense0 = nn.Linear(30, neurons)
    initializer(self.dense0.weight)
    self.activation0 = activation
    self.dense1 = nn.Linear(neurons, neurons)
    initializer(self.dense1.weight)
    self.activation1 = activation
    self.dense2 = nn.Linear(neurons, 1)
    initializer(self.dense2.weight)
    self.output = nn.Sigmoid()

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)
    X = self.dense1(X)
    X = self.activation1(X)
    X = self.dense2(X)
    X = self.output(X)
    return X

In [ ]:
classificado_sklearn = NeuralNetBinaryClassifier(module=classificador_torch,
                                                 lr= 0.001,
                                                 optimizer__weight_decay =0.0001,
                                                 train_split=False)


In [ ]:
parms = {'batch_size': [10, 30],
         'max_epochs': [50, 100],
         'optimizer': [torch.optim.Adam],#torch.optim.SGD
         'criterion': [torch.nn.BCELoss], #torch.nn.HingeEmbeddingLoss],
         'module__activation': [F.relu],# F.tanh],
         'module__neurons': [16],#8
         'module__initializer': [torch.nn.init.uniform_], #torch.nn.init.normal_]
         }

In [ ]:
grid_search = GridSearchCV(estimator=classificado_sklearn, param_grid=parms, scoring= 'accuracy', cv = 2)
grid_search = grid_search.fit(previsores,classes)


  epoch    train_loss     dur
-------  ------------  ------
      1       10.3130  0.1330
      2       10.3130  0.0391
      3       10.3130  0.0381
      4       10.3130  0.0395
      5       10.3130  0.0450
      6       10.3130  0.0382
      7       10.3130  0.0378
      8       10.3130  0.0416
      9       10.3130  0.0378
     10       10.3130  0.0392
     11       10.3130  0.0394
     12       10.3130  0.0385
     13       10.3130  0.0386
     14       10.3130  0.0387
     15       10.3130  0.0383
     16       10.3130  0.0438
     17       10.3130  0.0415
     18       10.3130  0.0378
     19       10.3130  0.0381
     20       10.3130  0.0388
     21       10.3130  0.0385
     22       10.3130  0.0399
     23       10.3130  0.0388
     24       10.3130  0.0391
     25       10.3130  0.0408
     26       10.3130  0.0439
     27       10.3130  0.0376
     28       10.3130  0.0384
     29       10.3130  0.0387
     30       10.3130  0.0384
     31       10.3130  0.0384
     32   

In [ ]:
mp = grid_search.best_params_
mpr = grid_search.best_score_

In [ ]:
mp

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCELoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.relu>,
 'module__initializer': <function torch.nn.init.uniform_>,
 'module__neurons': 16,
 'optimizer': torch.optim.adam.Adam}

In [ ]:
mpr

0.7436125525080306